In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt

import numpy as np
import matplotlib as mpl
from scipy import stats


from scipy.interpolate import interp1d
import pandas as pd


import pickle

from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import sklearn
from sklearn.model_selection import GridSearchCV
import itertools
import warnings


In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/

# Read all simulation results, train parameter scaler

In [4]:
folder='./../../../DataAndScripts/unstructured_scripts/simulation_results/'
results=np.loadtxt(folder+'results_0.txt');
results=np.concatenate((results,np.loadtxt(folder+'results_1.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_2.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_3.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_4.txt')));
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
tau_decay_all=results[:,26]
max_decay_all=results[:,27]
sim_decay_all=tau_decay_all/max_decay_all

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=np.zeros(np.shape(moments_of_r_sim_all))
Y[:,:]=moments_of_r_sim_all[:,:]
Y[:,4]=np.sign(Y[:,4])*np.sqrt(np.abs(Y[:,4]))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train) 

with open('./../../../DataAndScripts/unstructured_scripts/model_files/scaler.pkl','wb') as f:
    pickle.dump(scaler,f)


In [10]:
import sys

sys.path.insert(0, './../../../DataAndScripts/unstructured_scripts/')
import functions_optimal as fun

print('Create dataset')
data_mice=np.loadtxt('./../../../DataAndScripts/experimental_data/Mice_with_trials.txt');
data_monkeys=np.loadtxt('./../../../DataAndScripts/experimental_data/Monkeys_with_trials.txt');
data_both_species=[data_mice,data_monkeys]

dataset_mouse,Con_mouse,nCon_mouse=fun.build_dataset(data_both_species[0])
dataset_monkey,Con_monkey,nCon_monkey=fun.build_dataset(data_both_species[1])


Create dataset


In [12]:
dataset_mouse.shape

(6, 7, 2)

In [13]:
dataset_monkey.shape

(6, 6, 2)

In [15]:
np.concatenate((dataset_mouse,dataset_monkey),1).shape

(6, 13, 2)

# Design fits

## Write simulation parameters with set J and CVopto

In [6]:
Possible_idx_species=[0,1]
Possible_log10_CVopto=np.linspace(-1,1,20)
Possible_log10_J=np.linspace(-5,-2.5,20)
nRep=101

            
count=0;
for idx_species in [0,1,2]:
    for idx_S in range(len(Possible_log10_CVopto)):
        log10_CVopto=Possible_log10_CVopto[idx_S]
        for idx_J in range(len(Possible_log10_J)):
            log10_J=Possible_log10_J[idx_J]
            data=np.ones((1,4));
            data[:,0]=idx_species
            data[:,1]=log10_CVopto
            data[:,2]=log10_J
            data[:,3]=nRep

            if count==0:
                DATA=data;
            if count>0:
                DATA=np.vstack((DATA,data));
            count=count+1

len(DATA)

df = pd.DataFrame(DATA,columns=['idx_species','log10_CVopto','log10_J','nRep'])
file_name='./../../../DataAndScripts/unstructured_scripts/simulation_files/simulation_param_Fixed_J_and_CVopto.txt'
df.to_csv(file_name, sep='\t')

## Write simulation parameters with set J and CVopto

In [7]:
Possible_idx_species=[0,1]
possible_g_E=np.linspace(3,10,20)

            
count=0;
for idx_species in [0,1,2]:
    for idx_g_E in range(len(possible_g_E)):
        g_E=possible_g_E[idx_g_E];
        possible_g_I=np.arange(2,g_E,np.diff(possible_g_E)[0])
        for idx_g_I in range(len(possible_g_I)):
            g_I=possible_g_I[idx_g_I];

            data=np.ones((1,4));
            data[:,0]=idx_species
            data[:,1]=g_E
            data[:,2]=g_I
            data[:,3]=nRep

            if count==0:
                DATA=data;
            if count>0:
                DATA=np.vstack((DATA,data));
            count=count+1

print(len(DATA))

df = pd.DataFrame(DATA,columns=['idx_species','g_E','g_I','nRep'])
file_name='./../../../DataAndScripts/unstructured_scripts/simulation_files/simulation_param_Fixed_gs.txt'
df.to_csv(file_name, sep='\t')

750


# Train perceptron to predict optogenetic response statistics

In [3]:
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
sim_decay_all=results[:,26]/results[:,27]

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=np.zeros(np.shape(moments_of_r_sim_all))
Y[:,:]=moments_of_r_sim_all[:,:]
Y[:,4]=np.sign(Y[:,4])*np.sqrt(np.abs(Y[:,4]))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

# Use optimization in the plot reults file to find best network structure
mlp_regressor  = MLPRegressor(random_state=123,
                          activation='relu',
                          hidden_layer_sizes=(100, 150, 50),verbose=True)
mlp_regressor.fit(X_train, Y_train)

Y_preds = mlp_regressor.predict(X_test)

#print(Y_preds[:5])
#print(Y_test[:5])

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, Y_train))

Iteration 1, loss = 299.53041492
Iteration 2, loss = 107.43174408
Iteration 3, loss = 83.80918557
Iteration 4, loss = 70.72885762
Iteration 5, loss = 62.25586526
Iteration 6, loss = 55.92509836
Iteration 7, loss = 51.23740731
Iteration 8, loss = 47.23828537
Iteration 9, loss = 44.97432679
Iteration 10, loss = 42.60873252
Iteration 11, loss = 41.01595299
Iteration 12, loss = 39.82127341
Iteration 13, loss = 37.86555349
Iteration 14, loss = 37.38491539
Iteration 15, loss = 36.38159852
Iteration 16, loss = 35.49293117
Iteration 17, loss = 34.42527725
Iteration 18, loss = 34.20142608
Iteration 19, loss = 34.04286134
Iteration 20, loss = 32.58394463
Iteration 21, loss = 32.18017562
Iteration 22, loss = 32.30057694
Iteration 23, loss = 31.65292312
Iteration 24, loss = 31.38496213
Iteration 25, loss = 30.60763314
Iteration 26, loss = 30.37835711
Iteration 27, loss = 30.02986226
Iteration 28, loss = 29.33444089
Iteration 29, loss = 29.41089067
Iteration 30, loss = 29.04719513
Iteration 31, los

/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Test R^2 Score : 0.953
Training R^2 Score : 0.957


In [ ]:
with open('./../../../DataAndScripts/unstructured_scripts/model_files/model.pkl','wb') as f:
    pickle.dump(mlp_regressor,f)

# Train perceptron to predict chaos

In [5]:
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
sim_decay_all=results[:,26]/results[:,27]

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=tau_decay_all
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

# Use optimization in the plot reults file to find best network structure
mlp_regressor_chaos  = MLPRegressor(random_state=123,
                          activation='relu',
                          hidden_layer_sizes=(100, 150, 50),verbose=True)
mlp_regressor_chaos.fit(X_train, Y_train)

Y_preds = mlp_regressor_chaos.predict(X_test)

#print(Y_preds[:5])
#print(Y_test[:5])

print('Test R^2 Score : %.3f'%mlp_regressor_chaos.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor_chaos.score(X_train, Y_train))

Iteration 1, loss = 0.05581922
Iteration 2, loss = 0.03564423
Iteration 3, loss = 0.03293018
Iteration 4, loss = 0.03128070
Iteration 5, loss = 0.03046225
Iteration 6, loss = 0.02979165
Iteration 7, loss = 0.02927413
Iteration 8, loss = 0.02881059
Iteration 9, loss = 0.02867931
Iteration 10, loss = 0.02840478
Iteration 11, loss = 0.02815954
Iteration 12, loss = 0.02801896
Iteration 13, loss = 0.02782693
Iteration 14, loss = 0.02767829
Iteration 15, loss = 0.02754842
Iteration 16, loss = 0.02750537
Iteration 17, loss = 0.02731747
Iteration 18, loss = 0.02725669
Iteration 19, loss = 0.02713766
Iteration 20, loss = 0.02700899
Iteration 21, loss = 0.02703340
Iteration 22, loss = 0.02688194
Iteration 23, loss = 0.02675279
Iteration 24, loss = 0.02678707
Iteration 25, loss = 0.02672548
Iteration 26, loss = 0.02666041
Iteration 27, loss = 0.02657491
Iteration 28, loss = 0.02644893
Iteration 29, loss = 0.02646579
Iteration 30, loss = 0.02644816
Iteration 31, loss = 0.02640419
Iteration 32, los

/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Test R^2 Score : 0.917
Training R^2 Score : 0.922


In [ ]:
with open('./../../../DataAndScripts/unstructured_scripts/model_files/chaos_model.pkl','wb') as f:
    pickle.dump(mlp_regressor_chaos,f)